In [1]:
from pandas import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [2]:
df = pd.read_csv('BanglaNames.csv')
df = df.drop_duplicates(subset="Name")
df.head()

Id    Name  Gender
0   0  fairuz  female
1   1  maliha  female
2   2  mehedi    male
3   3   sohan    male
4   4  shatil    male

In [3]:
# Check if the name ends in vowel
def checkVowelEnd(name):
    if name[-1] in "aeiou":
        return "Vowel End"
    else:
        return "Consonant End"

In [4]:
df["Vowel/Consonant End"] = df["Name"].apply(checkVowelEnd)
df.head()

Id    Name  Gender Vowel/Consonant End
0   0  fairuz  female       Consonant End
1   1  maliha  female           Vowel End
2   2  mehedi    male           Vowel End
3   3   sohan    male       Consonant End
4   4  shatil    male       Consonant End

In [5]:
def checkGender(gender):
    if gender == "female":
        return 0
    else:
        return 1
    
df["Gender Value"] = df["Gender"].apply(checkGender)

df.head()

Id    Name  Gender Vowel/Consonant End  Gender Value
0   0  fairuz  female       Consonant End             0
1   1  maliha  female           Vowel End             0
2   2  mehedi    male           Vowel End             1
3   3   sohan    male       Consonant End             1
4   4  shatil    male       Consonant End             1

In [6]:
def compare(group):
    return df.groupby([group])["Gender Value"].sum()*100/df.groupby([group])["Gender Value"].count()

In [7]:
df.groupby(["Vowel/Consonant End"])['Gender Value'].count()

Vowel/Consonant End
Consonant End    562
Vowel End        447
Name: Gender Value, dtype: int64

In [8]:
print (len(df))
# > 93889 = 43635 + 50254

1009


In [9]:
female_names = sum(df.groupby(["Vowel/Consonant End"])["Gender Value"].sum())
all_names = df.groupby(["Gender"])["Gender Value"].count()
print (all_names)
print ("\nBoth are equal? %s" % str(female_names == all_names["female"]))

Gender
female    573
male      436
Name: Gender Value, dtype: int64

Both are equal? False


In [10]:
print(df.groupby(["Vowel/Consonant End"])["Gender Value"].sum()*100/df.groupby(["Vowel/Consonant End"])["Gender Value"].count())

Vowel/Consonant End
Consonant End    66.903915
Vowel End        13.422819
Name: Gender Value, dtype: float64


In [11]:
print(compare("Vowel/Consonant End"))

Vowel/Consonant End
Consonant End    66.903915
Vowel End        13.422819
Name: Gender Value, dtype: float64


In [12]:
def vowelConsonantStart(name):
    if name[0] in "aeiou":
        return "Vowel Start"
    else:
        return "Consonant Start"

df["Vowel/Consonant Start"] = df["Name"].apply(vowelConsonantStart)

print("\n Comparison => %s", compare("Vowel/Consonant Start"))

df.head()


 Comparison => %s Vowel/Consonant Start
Consonant Start    41.016109
Vowel Start        51.980198
Name: Gender Value, dtype: float64


Id    Name  Gender Vowel/Consonant End  Gender Value Vowel/Consonant Start
0   0  fairuz  female       Consonant End             0       Consonant Start
1   1  maliha  female           Vowel End             0       Consonant Start
2   2  mehedi    male           Vowel End             1       Consonant Start
3   3   sohan    male       Consonant End             1       Consonant Start
4   4  shatil    male       Consonant End             1       Consonant Start

In [13]:
def shortLongName(name):
    if len(name) < 6:
        return "Short"
    else:
        return "Long"

df["Short/Long Name"] = df["Name"].apply(shortLongName)
print(compare("Short/Long Name"))
df.head()

Short/Long Name
Long     39.304348
Short    48.387097
Name: Gender Value, dtype: float64


Id    Name  Gender Vowel/Consonant End  Gender Value Vowel/Consonant Start  \
0   0  fairuz  female       Consonant End             0       Consonant Start   
1   1  maliha  female           Vowel End             0       Consonant Start   
2   2  mehedi    male           Vowel End             1       Consonant Start   
3   3   sohan    male       Consonant End             1       Consonant Start   
4   4  shatil    male       Consonant End             1       Consonant Start   

  Short/Long Name  
0            Long  
1            Long  
2            Long  
3           Short  
4            Long

In [14]:
training_data = df[["Gender Value", "Vowel/Consonant Start", "Short/Long Name", "Vowel/Consonant End"]]
training_data.head()

Gender Value Vowel/Consonant Start Short/Long Name Vowel/Consonant End
0             0       Consonant Start            Long       Consonant End
1             0       Consonant Start            Long           Vowel End
2             1       Consonant Start            Long           Vowel End
3             1       Consonant Start           Short       Consonant End
4             1       Consonant Start            Long       Consonant End

In [15]:
def reprCategory(column):
    column = column.astype("category")
    return column.cat.codes
    
training_data[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]] = training_data[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]].apply(reprCategory)

training_data.head()

C:\Users\IMF\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Gender Value  Vowel/Consonant Start  Short/Long Name  Vowel/Consonant End
0             0                      0                0                    0
1             0                      0                0                    1
2             1                      0                0                    1
3             1                      0                1                    0
4             1                      0                0                    0

In [16]:
train, test = train_test_split(training_data, test_size = 0.20)

In [17]:
# Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB
MultinomialNB_clf = MultinomialNB()
MultinomialNB_clf.fit(train[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]], train["Gender Value"])
predictions = MultinomialNB_clf.predict(test[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]])
accuracy_score(test["Gender Value"], predictions)

0.7376237623762376

In [18]:
#from sklearn.metrics import accuracy_score
#import numpy as np
#import matplotlib.pyplot as plt
#from matplotlib.colors import ListedColormap

#fig = plt.figure(figsize=(15,20))
#def run_model(model, alg_name, plot_index):
 #   # build the model on training data
  #  model.fit(train[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]], train["Gender Value"].values.reshape(-1,))
#
 #   # make predictions for test data
  #  y_pred = model.predict(test[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]])
   # # calculate the accuracy score
    #accuracy =  accuracy_score(test["Gender Value"], y_pred) * 100
#
 #   # Compare the prediction result with ground truth
  #  color_code = {'0':'red', '1':'blue'}
#
 #   # plt.figure(plot_index)
  #  ax = fig.add_subplot(2,2,plot_index) 
   # colors = [color_code[x] for x in test["Gender Value"].iloc[:,0]]
    #ax.scatter(test[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]].iloc[:,0], test[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]].iloc[:,2], color=colors, marker='.', label='Circle = Ground truth')
    #colors = [color_code[x] for x in y_pred]
    #ax.scatter(test[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]].iloc[:, 0], test[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]].iloc[:,2], color=colors, marker='o', facecolors='none', label='Dot = Prediction')

    #plt.axes([0.65, 0.65, 0.2, 0.2])
    #ax.legend(loc="lower right")
    ## manually set legend color to black
    #leg = plt.gca().get_legend()
    #leg.legendHandles[0].set_color('black')
    #leg.legendHandles[1].set_color('black')
    #leg.legendHandles[1].set_facecolors('none')

    #ax.set_title(alg_name + ". Accuracy: " + str(round(accuracy,4)))


In [19]:
#from sklearn.naive_bayes import MultinomialNB
#model = MultinomialNB()
#run_model(model, "MultinomialNB", 1)

In [20]:
# Dicision Tree Classifier
DecisionTreeClassifier_clf = DecisionTreeClassifier()
DecisionTreeClassifier_clf = DecisionTreeClassifier_clf.fit(train[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]], train["Gender Value"])
predictions = DecisionTreeClassifier_clf.predict(test[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]])
accuracy_score(test["Gender Value"], predictions)

0.7425742574257426

In [21]:
from sklearn.linear_model import LogisticRegression
logreg_clf = LogisticRegression()
logreg_clf.fit(train[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]], train["Gender Value"])
predictions = logreg_clf.predict(test[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]])
accuracy_score(test["Gender Value"], predictions)

C:\Users\IMF\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7425742574257426

In [22]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
SVC_model = SVC()
SVC_model.fit(train[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]], train["Gender Value"])
prediction = SVC_model.predict(test[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]])
accuracy_score(test["Gender Value"], prediction)

C:\Users\IMF\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.7425742574257426

In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
KNN_model = KNeighborsClassifier(n_neighbors=5)
KNN_model.fit(train[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]], train["Gender Value"])
predictions = KNN_model.predict(test[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]])
accuracy_score(test["Gender Value"], predictions)

0.7376237623762376

In [24]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_clf = LinearDiscriminantAnalysis()
lda_clf.fit(train[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]], train["Gender Value"])
predictions = lda_clf.predict(test[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]])
accuracy_score(test["Gender Value"], predictions)
#print('Accuracy of LDA classifier on training set: {:.2f}'
 #    .format(lda.score(X_train, y_train)))
#print('Accuracy of LDA classifier on test set: {:.2f}'
 #    .format(lda.score(X_test, y_test)))

0.7425742574257426

In [25]:
from sklearn.naive_bayes import GaussianNB
gnb_clf = GaussianNB()
gnb_clf.fit(train[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]], train["Gender Value"])
predictions = gnb_clf.predict(test[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]])
accuracy_score(test["Gender Value"], predictions)

0.7425742574257426

In [26]:
#clf = clf.fit(train[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]], train["Gender Value"])

In [31]:
#clf

NameError: name 'clf' is not defined

In [28]:
#clf.feature_importances_

In [29]:
#predictions = clf.predict(test[["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"]])
#accuracy_score(test["Gender Value"], predictions)

In [32]:
with open("decidenames.dot", "w") as dot_file:
    dot_file = export_graphviz(DecisionTreeClassifier_clf,
                            feature_names=["Vowel/Consonant End", "Short/Long Name", "Vowel/Consonant Start"], out_file=dot_file)